# Required imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import KFold
from cnn_2d_model.CNN_LSTM_model import HelplessnessClassifier as CNN_LSTM_Classifier
from cnn_3d_model.model import HelplessnessClassifier as CNN_3D_Classifier
from pre_trained_transformer_model.model import create_swin3d_t_model_training

device = 'cpu'
if torch.backends.mps.is_available():
    device = 'mps' # Apple Silicon 
if torch.cuda.is_available():
    device = 'cuda' # Nvidia GPU
print(f'Using device: {device}')

Using device: cuda


# Custom dataset for cross-validation

In [2]:
class HelplessnessVideoDataset(Dataset):
    def __init__(self, root_dir, train_transform=None, test_transform=None):
        self.root_dir = root_dir
        self.train_transform = train_transform
        self.test_transform = test_transform
        self.train = True
        self.video_folders = []

        # Read all video sequences in the extracted frames folders
        categories = ['extreme-helpless', 'little_helplessness', 'no-helpless']
        for category in categories:
            category_dir = os.path.join(root_dir, category)
            if not os.path.exists(category_dir):
                print(f"Warning: Category folder {category_dir} does not exist.")
                continue
            for video_folder in sorted(os.listdir(category_dir)):
                video_path = os.path.join(category_dir, video_folder)
                if os.path.isdir(video_path):
                    self.video_folders.append(video_path)

    def __len__(self):
        return len(self.video_folders)

    def __getitem__(self, index):
        video_path = self.video_folders[index]

        # Each video is a sequence of frames, so need to get each frame
        frame_files = sorted(os.listdir(video_path))
        sequence = []
        random_state = torch.get_rng_state()
        for frame_name in frame_files:
            frame_path = os.path.join(video_path, frame_name)
            frame = Image.open(frame_path)
            if self.train_transform and self.test_transform:
                # To allow augmentation, we need to apply the same "random" transformation to each frame
                torch.set_rng_state(random_state)
                frame = self.train_transform(frame) if self.train else self.test_transform(frame)
            sequence.append(frame)

        # Convert the sequence from list of tensors to (sequence_length, channels, height, width) tensor
        sequence = torch.stack(sequence)
        sequence = torch.transpose(sequence, 0, 1)  # REMOVE THIS IF YOU NEED THE SEQUENCE_LENGTH AND CHANNELS DIMENSIONS SWITCHED!

        # Retrieve the level of helplessness label from path of video
        split_path = video_path.split(os.sep)  # Changed '/' to os.sep for cross-platform compatibility

        # Ensure that split_path has at least 2 components
        if len(split_path) >= 2:
            category = split_path[-2]  # category folder is second last in path
        else:
            raise ValueError(f"Path structure issue: {video_path}")

        label = -1
        if category == 'no-helpless':
            label = 0
        elif category == 'little_helplessness':
            label = 1
        elif category == 'extreme-helpless':
            label = 2

        return sequence, label

# 2D CNN-LSTM 5-fold Cross Validation

In [3]:
# Create the transforms and dataset for 2D CNN-LSTM

train_transform = transforms.Compose([
    transforms.RandomResizedCrop((112, 112), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),  # converts PIL Image in L mode to tensor shape (1, H, W) with pixel values [0,1]
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = HelplessnessVideoDataset('../data/train', train_transform=train_transform, test_transform=test_transform)

In [7]:
# to do 5-fold cross validation, we are using the approach shown here: https://saturncloud.io/blog/how-to-use-kfold-cross-validation-with-dataloaders-in-pytorch/

# Initialize how many folds we want for cross-validation
# Using KFold from scikit-learn for this: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kf = KFold(n_splits=5, shuffle=True)
average_f1score = 0

# Loop through each fold, training the model and reporting results
for fold, (train_index, test_index) in enumerate(kf.split(train_dataset)):
    print(f'Fold {fold + 1}')
    print('------')

    # Create data loaders for the current fold
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=4,
        sampler=torch.utils.data.SubsetRandomSampler(train_index),
        num_workers=2
    )

    test_loader = DataLoader(
        dataset=train_dataset,
        batch_size=4,
        sampler=torch.utils.data.SubsetRandomSampler(test_index),
        num_workers=2
    )

    # Initialize the model, loss criterion, optimizer and number of epoches
    model = CNN_LSTM_Classifier(num_classes=3).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    num_epochs = 20

    # Train the model for the given epoches on the train_loader for this fold
    for epochs in tqdm(range(num_epochs)):
        model.train()
        train_dataset.train = True # ensure train transforms on used on the data

        for i, (sequences, labels) in enumerate(train_loader):
            sequences = torch.transpose(sequences, 1, 2) # need to transpose the sequence length and channel dimensions
            sequences, labels = sequences.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(sequences)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()

            # Optimize the model
            optimizer.step()

    # Evaluate the model on the test_loader for this fold
    model.eval()
    train_dataset.train = False # ensure test transforms on used on the data
    predictions = []
    ground_truth = []
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (sequences, labels) in enumerate(test_loader):
            sequences = torch.transpose(sequences, 1, 2) # need to transpose the sequence length and channel dimensions
            sequences, labels = sequences.to(device), labels.to(device)

            # Forward pass
            outputs = model(sequences)

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Store predictions and ground truth labels in lists
            predictions.extend(predicted.detach().cpu().tolist())
            ground_truth.extend(labels.detach().cpu().tolist())
    
    # Using scikit-learn, print the F1 score of this fold: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
    from sklearn.metrics import f1_score
    fold_f1score = f1_score(ground_truth, predictions, average='weighted') * 100
    fold_accuracy = 100 * correct / total
    print(f'Fold F1 Score: {fold_f1score:.2f}%, Fold Accuracy: {fold_accuracy:.2f}%\n')
    average_f1score += fold_f1score

average_f1score /= 5
print(f'Average F1 Score across all folds: {average_f1score:.2f}%')

Fold 1
------
[DEBUG] CNN feature_dim = 25088


  0%|          | 0/20 [00:05<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 32980, 28852) exited unexpectedly